<h1 style="text-align: center;"> SUBSTANTIA NIGRA (SN) - MEAN INTENSITY EXTRACTION </h1>

The following code was used to extract mean signal intensities in SNpc (left, right and both hemispheres)

In [ ]:
import os
import pandas as pd
import subprocess

In [ ]:
#defining base directory and SNpc mask names
base_dir = "/hus/home/oliwin/data/study_name"
mask_sn_l = f"{base_dir}/SN_L_mask.nii.gz"
mask_sn_r = f"{base_dir}/SN_R_mask.nii.gz"
SN_mask = f"{base_dir}/SN_mask.nii.gz"

# Fusing left and right hemispheres of SN
!fslmaths {mask_sn_l} -add {mask_sn_r} {SN_mask}

In [ ]:
# dictionary to store participant-wise data
patients_data = {}

# loop through subdirectories 'FSL' in base_dir (recursively)
for root, dirs, files in os.walk(base_dir):
    if "FSL" in dirs:
        fsl_dir = os.path.join(root, "FSL")
        
        # extract participant number from the folder name
        patient_number = os.path.basename(root)
        
        # loop through NIfTI files in FSL directory
        for filename in os.listdir(fsl_dir):
            # process files with "mni", "nii.gz"
            if (
                "mni" in filename
                and "mask" not in filename
                and "new" not in filename
                and "flirt" not in filename
                and "flirt_masked" not in filename
                and not filename.endswith(".mat")
                and filename.endswith(".nii.gz")
            ):
                input_file = os.path.join(fsl_dir, filename)

                
                print(f"Processing file: {input_file}")

                # loop through the SNpc masks (left, right and combined) - each MRI file is processed three times
                for mask_file in [SN_mask, mask_sn_r, mask_sn_l]:
                    try:
                        # use FLIRT to perform the registration of SNpc masks
                        flirt_output = f"{input_file}_flirt_{os.path.basename(mask_file).split('.')[0]}"
                        flirt_matrix = f"{flirt_output}.mat"
                        flirt_command = f"flirt -in {input_file} -ref {mask_file} -out {flirt_output} -omat {flirt_matrix}"
                        process_flirt = subprocess.run(flirt_command, shell=True, capture_output=True, text=True)

                        if process_flirt.returncode != 0:
                            print(f"Error applying FLIRT to {filename} with mask {mask_file}: {process_flirt.stderr}")
                            continue  # continue with the next mask for the same file

                        # use fslmaths to apply the masks and threshold negative values to zero
                        flirt_masked = f"{flirt_output}_masked"
                        fslmaths_command = f"fslmaths {flirt_output} -mas {mask_file} -thr 0 {flirt_masked}"
                        process_fslmaths = subprocess.run(fslmaths_command, shell=True, capture_output=True, text=True)

                        if process_fslmaths.returncode != 0:
                            print(f"Error applying fslmaths to {filename} with mask {mask_file}: {process_fslmaths.stderr}")
                            continue  # continue with the next mask for the same file

                        # use fslstats to extract mean intensity within the region
                        stats_command = f"fslstats {flirt_masked} -M"  # -M for Mean Intensity 
                        process_stats = subprocess.run(stats_command, shell=True, capture_output=True, text=True)

                        if process_stats.returncode != 0:
                            print(f"Error processing {filename}: {process_stats.stderr}")
                            continue  # continue with the next mask for the same file

                        
                        stats = process_stats.stdout.strip().split()
                        mean_intensity = float(stats[0]) 

                        # remove intermediate files and temporary NIfTI outputs
                        try:
                            for file_to_delete in [flirt_matrix, flirt_output + ".nii.gz", flirt_masked + ".nii.gz"]:
                                if os.path.exists(file_to_delete):
                                    os.remove(file_to_delete)
                                else:
                                    print(f"File not found: {file_to_delete}")
                        except OSError as e:
                            print(f"Error deleting files for {filename}: {e}")

                        # add the data to participant's entry
                        if patient_number not in patients_data:
                            patients_data[patient_number] = {}

                        mask_name = os.path.basename(mask_file).split('.')[0]  
                        filename_base = filename.replace('.nii.gz', '')  
                        mask_key = f'{mask_name}_{filename_base}'

                        patients_data[patient_number][mask_key] = mean_intensity

                    except Exception as e:
                        print(f"Error processing {filename} with mask {mask_file} for patient {patient_number}: {e}")
                        continue  # continue to the next mask or next file

# preparing the dataframe to store mean intensities
header = ['Patient Number']
all_keys = set()
for intensities in patients_data.values():
    all_keys.update(intensities.keys())
header.extend(sorted(all_keys))  

output_data = []
for patient_number, intensities in patients_data.items():
    row = [patient_number]
    for col in header[1:]:
        row.append(intensities.get(col, None))  # use None for missing values
    output_data.append(row)

# creating the dataframe
output_df = pd.DataFrame(output_data, columns=header)

# saving the dataframe as csv for further analysis
output_file = 'mri_data_sn.csv'  
output_df.to_csv(output_file, index=False)
print(f"Transformed CSV saved to {output_file}")

Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_meSWIM_HPF_filled_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_meSWIM_filled_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_meSWIM_filled_MIP_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_mpSWIM_ECHO-3_e3_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_pSWIM_ECHO-3_e3_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simCSF_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simFLAIR_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simGM_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simWM_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_sim_GRE_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/st

Traceback (most recent call last):
  File "/hus/home/oliwin/.local/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/tmp/ipykernel_833046/56921799.py", line 106, in <module>
    output_df.to_csv(output_file, index=False)
  File "/hus/home/oliwin/.local/lib/python3.12/site-packages/pandas/util/_decorators.py", line 333, in wrapper
  File "/hus/home/oliwin/.local/lib/python3.12/site-packages/pandas/core/generic.py", line 3967, in to_csv
  File "/hus/home/oliwin/.local/lib/python3.12/site-packages/pandas/io/formats/format.py", line 987, in to_csv
ModuleNotFoundError: No module named 'pandas.io.formats.csvs'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/hus/home/oliwin/.local/lib/python3.12/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exce

In [4]:
print(output_df)

   Patient Number  SN_L_mask_STAGE_CROWN_PD_MAP_reg_mni  \
0        0000D71C                            611.134328   
1        0000D2D0                            682.961905   
2        0000997A                            699.316239   
3        00008904                            629.702703   
4        00008289                            626.151685   
..            ...                                   ...   
62       00005018                            673.416309   
63       00004943                            633.224670   
64       000038BE                            655.110599   
65       00004468                            607.886364   
66       00002478                            629.067265   

    SN_L_mask_STAGE_CROWN_PD_MAPa_reg_mni  \
0                                     NaN   
1                                     NaN   
2                                     NaN   
3                                     NaN   
4                                     NaN   
..                   

In [ ]:
# saving the mean intensities that were extracted before server crash to csv
output_file = 'mri_data_sn.csv'  
output_df.to_csv(output_file, index=False)
print(f"Transformed CSV saved to {output_file}")

Transformed CSV saved to mri_data_sn.csv


The Linux server crashed during the first extraction. The participants that were missed were identified by cross-referencing the patient IDs from the CSV file containing WM intensities. The extraction was then performed for these missing participants. 

In [ ]:
# first round of extraction from SN cross referenced with WM extraction csv to find missing participants
sn_file = "mri_data_sn.csv"  
white_file = "mri_data_white.csv" 

sn_df = pd.read_csv(sn_file)
white_df = pd.read_csv(white_file)

# extracting 'Patient Number' column from both files
sn_patients = set(sn_df['Patient Number'])
white_patients = set(white_df['Patient Number'])

# identifying participants in WM csv file, but not in SN csv
missing_patients = white_patients - sn_patients

print("Missing patients:")
for patient in sorted(missing_patients):
    print(patient)


Missing patients:
00000EAE
000030C7
0000868E
000090DC
0000991B
0000AF0D
0000B5CD
0000C2A9
0000C65E
0000C788
0000CC09
0000CFCD
0000DAD2
0000DEB2


In [ ]:
# define the list of missing participants
missing_patients = [
    "00000EAE", "000030C7", "0000868E", "000090DC", "0000991B", 
    "0000AF0D", "0000B5CD", "0000C2A9", "0000C65E", "0000C788", 
    "0000CC09", "0000CFCD", "0000DAD2", "0000DEB2", "00002478"
]


patients_data = {}

for root, dirs, files in os.walk(base_dir):
    if "FSL" in dirs:
        fsl_dir = os.path.join(root, "FSL")
        
        # extract participant number 
        patient_number = os.path.basename(root)

        # process only the specified missing participants
        if patient_number not in missing_patients:
            continue

        # loop through NIfTI files in the FSL directory
        for filename in os.listdir(fsl_dir):
            # process only files with "mni", "nii.gz"
            if (
                "mni" in filename
                and "mask" not in filename
                and "new" not in filename
                and "flirt" not in filename
                and "flirt_masked" not in filename
                and not filename.endswith(".mat")
                and filename.endswith(".nii.gz")
            ):
                input_file = os.path.join(fsl_dir, filename)

                print(f"Processing file: {input_file}")

                # loop through the SNpc masks (left, right and combined) - each MRI file is processed three times  
                for mask_file in [SN_mask, mask_sn_r, mask_sn_l]:
                    try:
                        # use FLIRT to perform the registration of SNpc masks
                        flirt_output = f"{input_file}_flirt_{os.path.basename(mask_file).split('.')[0]}"
                        flirt_matrix = f"{flirt_output}.mat"
                        flirt_command = f"flirt -in {input_file} -ref {mask_file} -out {flirt_output} -omat {flirt_matrix}"
                        process_flirt = subprocess.run(flirt_command, shell=True, capture_output=True, text=True)

                        if process_flirt.returncode != 0:
                            print(f"Error applying FLIRT to {filename} with mask {mask_file}: {process_flirt.stderr}")
                            continue  # continue with the next mask for same file

                        # use fslmaths to apply the masks and threshold negative values to zero
                        flirt_masked = f"{flirt_output}_masked"
                        fslmaths_command = f"fslmaths {flirt_output} -mas {mask_file} -thr 0 {flirt_masked}"
                        process_fslmaths = subprocess.run(fslmaths_command, shell=True, capture_output=True, text=True)

                        if process_fslmaths.returncode != 0:
                            print(f"Error applying fslmaths to {filename} with mask {mask_file}: {process_fslmaths.stderr}")
                            continue  # continue with the next mask for same file

                        # use fslstats to extract mean intensity within the region
                        stats_command = f"fslstats {flirt_masked} -M"  # -M for Mean Intensity 
                        process_stats = subprocess.run(stats_command, shell=True, capture_output=True, text=True)

                        if process_stats.returncode != 0:
                            print(f"Error processing {filename}: {process_stats.stderr}")
                            continue  # continue with the next mask for same file

                        
                        stats = process_stats.stdout.strip().split()
                        mean_intensity = float(stats[0])  

                        # remove intermediate files and temporary NIfTI outputs 
                        try:
                            for file_to_delete in [flirt_matrix, flirt_output + ".nii.gz", flirt_masked + ".nii.gz"]:
                                if os.path.exists(file_to_delete):
                                    os.remove(file_to_delete)
                                else:
                                    print(f"File not found: {file_to_delete}")
                        except OSError as e:
                            print(f"Error deleting files for {filename}: {e}")

                        # add data to the participant's entry
                        if patient_number not in patients_data:
                            patients_data[patient_number] = {}

                        mask_name = os.path.basename(mask_file).split('.')[0]  
                        filename_base = filename.replace('.nii.gz', '') 
                        mask_key = f'{mask_name}_{filename_base}'

                        patients_data[patient_number][mask_key] = mean_intensity

                    except Exception as e:
                        print(f"Error processing {filename} with mask {mask_file} for patient {patient_number}: {e}")
                        continue  # continue to next mask or next file

# preparing the dataframe to store mean intensities
header = ['Patient Number']
all_keys = set()
for intensities in patients_data.values():
    all_keys.update(intensities.keys())
header.extend(sorted(all_keys))  

output_data = []
for patient_number, intensities in patients_data.items():
    row = [patient_number]
    for col in header[1:]:
        row.append(intensities.get(col, None))  # use None for missing values
    output_data.append(row)

# creating the dataframe
output_df = pd.DataFrame(output_data, columns=header)

# saving the dataframe as csv file for further analysis
output_file = 'mri_data_sn_missing_patients.csv'
output_df.to_csv(output_file, index=False)
print(f"Transformed CSV saved to {output_file}")


Processing file: /hus/home/oliwin/data/study_name/00002478/FSL/mprage_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/00002478/FSL/STAGE_CROWN_PD_MAP_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/00002478/FSL/STAGE_CROWN_R2S_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/00002478/FSL/STAGE_CROWN_R2S_A2_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/00002478/FSL/STAGE_CROWN_T2S_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/00002478/FSL/STAGE_CROWN_T2S_A2_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/00002478/FSL/STAGE_CROWN_TRUE_PD_MAP_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/00002478/FSL/STAGE_CROWN_TRUE_PD_MAPa_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/00002478/FSL/STAGE_HPF_ECHO-3_e3_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/00002478/FSL/STAGE_KMAP_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study